In [ ]:
import os
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import clear_output
import seaborn as sns
import math
from collections import Counter
from scipy import stats

#### Variables

In [ ]:
action_legends = [
    'Drop Home Pheromone', 
    'Drop Food Pheromone', 
    'Follow Home Pheromone', 
    'Follow Food Pheromone',
    'Move Randomly'
]

In [ ]:
root = '../data/Condition_4/'
folders = os.listdir(root)

index = 18

folder = folders[index]
folder_path = root + folder + '/'

In [ ]:
simulation_data = {}

#### Read Simulation Parameters

In [ ]:
def read_params_data(folder_path):
    file_path = folder_path + 'params.json'
    with open(file_path) as readFile:
        return json.load(readFile)

In [ ]:
simulation_data['params'] = read_params_data(folder_path)

In [ ]:
antCount = simulation_data['params']['antCount']
actions = simulation_data['params']['actions']
stepsCount = simulation_data['params']['stepsCount']

# bin_size = 1000
# num_bins = math.ceil(stepsCount / bin_size)

num_bins = 500
bin_size = math.ceil(stepsCount / num_bins)

#### Read Agents Data

In [ ]:
def read_agents_data(folder_path, num_agents):
    agents_data = {}
    for ant in range(1, num_agents+1):
        agents_data[ant] = pd.read_pickle(folder_path + 'agent_' + str(ant) + '.pkl')    
    return agents_data

In [ ]:
simulation_data['agents'] = read_agents_data(folder_path, antCount)

#### Bin Data

In [ ]:
def generate_chunks(seq, size):
    for pos in range(0, len(seq), size):
        yield seq.iloc[pos:pos+size]

In [ ]:
def create_binned_actions(agent_df, actions, bin_size, num_bins):
    actions_df = agent_df.filter(['action'])

    binned_actions = {}
    for i in actions:
        binned_actions[i] = []
    
    for chunk in generate_chunks(actions_df, bin_size):
        action_count = chunk['action'].value_counts()
    
        for action in actions:
            try:
                binned_actions[action].append(action_count[action])
            except:
                binned_actions[action].append(0)
    
    for action in actions:
        binned_actions[i] = np.array(binned_actions[i])
    
    return binned_actions

In [ ]:
def create_binned_trips(agent_df, bin_size, num_bins):
    trips_df = agent_df.filter(['tripCompleted'])
    
    binned_trips = []

    for chunk in generate_chunks(trips_df, bin_size):
        trips_count = chunk['tripCompleted'].value_counts()
    
        try:
            binned_trips.append(trips_count[1])
        except:
            binned_trips.append(0)
    
    return np.array(binned_trips)

In [ ]:
def create_binned_epsilon(agent_df, bin_size, num_bins):
    epsilon_df = agent_df.filter(['epsilon'])
    
    binned_epsilon = []

    for chunk in generate_chunks(epsilon_df, bin_size):
        epsilon_value = chunk.iloc[int(0)]['epsilon']
        binned_epsilon.append(epsilon_value)

    return np.array(binned_epsilon)

In [ ]:
simulation_data['binned_actions'] = {}
simulation_data['binned_trips'] = {}
simulation_data['binned_epsilon'] = {}

for ant in range(1, antCount+1):
    simulation_data['binned_actions'][ant] = create_binned_actions(simulation_data['agents'][ant], actions, bin_size, num_bins)
    simulation_data['binned_trips'][ant] = create_binned_trips(simulation_data['agents'][ant], bin_size, num_bins)
    simulation_data['binned_epsilon'][ant] = create_binned_epsilon(simulation_data['agents'][ant], bin_size, num_bins)

In [ ]:
simulation_data['total_binned_actions'] = {}

for action in actions:
    simulation_data['total_binned_actions'][action] = np.zeros(num_bins)

simulation_data['total_binned_trips'] = np.zeros(num_bins)
simulation_data['total_binned_epsilon'] = simulation_data['binned_epsilon'][1]

In [ ]:
for ant in range(1, antCount+1):
    for action in actions:
        simulation_data['total_binned_actions'][action] += simulation_data['binned_actions'][ant][action]
        simulation_data['total_binned_trips'] += simulation_data['binned_trips'][ant]

In [ ]:
simulation_data['params']['version']

In [ ]:
print('Total food collected = ' + str(np.sum(simulation_data['total_binned_trips'])))
print('Mean food collected = ' + str(np.mean(simulation_data['total_binned_trips'])))
print('Median food collected = ' + str(np.median(simulation_data['total_binned_trips'])))

#### Pearson Correlation Coeffecient - Actions and Trips

In [ ]:
simulation_data['pcc'] = np.zeros((6,6))

for i in range(5):
    for j in range(5):
        if(i == j): continue
        a = simulation_data['total_binned_actions'][i]
        b = simulation_data['total_binned_actions'][j]
        rho = stats.pearsonr(a, b)[0]
        simulation_data['pcc'][i,j] = rho
        
for i in range(5):
    a = simulation_data['total_binned_actions'][i]
    b = simulation_data['total_binned_trips']
    rho = stats.pearsonr(a, b)[0]
    simulation_data['pcc'][i,5] = rho
    simulation_data['pcc'][5,i] = rho

#### Transition Matrix - Action to Action

In [ ]:
def caluculate_action_TM(actions):
    a = actions
    b = np.zeros((5,5), dtype='int32')

    for i in range(len(a)-1):
        if(a[i] == -1 or a[i+1] == -1): continue
        b[a[i]][a[i+1]] += 1
    
    return b

In [ ]:
simulation_data['action_TM'] = {}
for ant in range(1, antCount+1):
    actions_list = simulation_data['agents'][ant].loc[:, 'action'].tolist()
    simulation_data['action_TM'][ant] =  caluculate_action_TM(actions_list)

simulation_data['action_TM'][1]

simulation_data['action_TM']['total'] = simulation_data['action_TM'][1]
for ant in range(2, antCount+1):
    simulation_data['action_TM']['total'] = np.add(simulation_data['action_TM']['total'], simulation_data['action_TM'][ant])

#### Transition Matrix - State to Action

In [ ]:
def caluculate__state_action_TM(states, actions):
    a = actions
    s = states
    size = len(a)
    
    tm = np.zeros((16, 5), dtype = 'int32')
    for i in range(size):
        if(s[i] == None or a[i] == -1): continue
        idx = s[i][0]*4 + s[i][1]
        tm[idx][a[i]] += 1

    return tm

In [ ]:
simulation_data['state_action_TM'] = {}

for ant in range(1, antCount+1):
    actions_list = simulation_data['agents'][ant].loc[:, 'action'].tolist()
    states_list = simulation_data['agents'][ant].loc[:, 'state'].tolist()
    simulation_data['state_action_TM'][ant] =  caluculate__state_action_TM(states_list, actions_list)

simulation_data['state_action_TM'][1]

simulation_data['state_action_TM']['total'] = simulation_data['state_action_TM'][1]
for ant in range(2, antCount+1):
    simulation_data['state_action_TM']['total'] = np.add(simulation_data['state_action_TM']['total'], simulation_data['state_action_TM'][ant])

#### Transition Matrix - State to State

In [ ]:
def get_state_TM(states):
    s = states
    size = len(s)
    
    tm = np.zeros((16, 16), dtype = 'int32')
    for i in range(size-1):
        if(s[i] == None or s[i+1] == None): continue
        idx = s[i][0]*4 + s[i][1]
        idx_next = s[i+1][0]*4 + s[i+1][1]
        tm[idx][idx_next] += 1

    return tm

In [ ]:
simulation_data['state_TM'] = {}

for ant in range(1, antCount+1):
    states_list = simulation_data['agents'][ant].loc[:, 'state'].tolist()
    simulation_data['state_TM'][ant] =  get_state_TM(states_list)

simulation_data['state_TM']['total'] = simulation_data['state_TM'][1]
for ant in range(2, antCount+1):
    simulation_data['state_TM']['total'] = np.add(simulation_data['state_TM']['total'], simulation_data['state_TM'][ant])

### Plotting

In [ ]:
binned_actions = simulation_data['total_binned_actions']
binned_trips = simulation_data['total_binned_trips']

total_actions = np.zeros(len(actions))
for action in actions:
    total_actions[action] = np.sum(binned_actions[action])

In [ ]:
################################################################
# Actions and Trips Stacked Plot

fig, axes = plt.subplots(2, 1, figsize = (15, 10))

x = np.array(range( len(binned_trips) )) * bin_size
y_actions = []
for action in actions:
    y_actions.append( binned_actions[action] )

axes[0].stackplot(x, y_actions, labels = action_legends)
# axes[0].title.set_text('Actions')
axes[0].legend(loc='upper left')

y_trips = binned_trips

axes[1].plot(x, y_trips, label ='Successful trips per agent for ' +str(bin_size)+ ' time steps')
# axes[1].title.set_text('Successful trip')
axes[1].legend(loc='upper left')

fig.suptitle('Actions and Trips across all agents')

fig.tight_layout()
fig.savefig(root + folder + '/' +folder+ '_actions_trips.jpg', bbox_inches='tight', dpi=250)

In [ ]:
################################################################
# Histogram of trips

plt.figure(figsize=(10,5))

temp = simulation_data['total_binned_trips']
sns_plot = sns.histplot(temp)
plt.title('Histogram of trips across all agents')

plt.savefig(root + folder + '/' +folder+ '_histogram_trips.jpg', bbox_inches='tight', dpi=250)

In [ ]:
################################################################
# Bar plot of number of actions

fig, ax = plt.subplots(figsize=(15,5))

ax.bar(x = action_legends, height = total_actions)
ax.set_title('Number of actions across all agents')

fig.tight_layout()
fig.savefig(root + folder + '/' +folder+ '_action_count.jpg', bbox_inches='tight', dpi=250)

In [ ]:
states_count = {}

for i in range(4):
    for j in range(4):
        states_count[(i,j)] = 0

for ant in range(1, antCount+1):
    d = simulation_data['agents'][ant]['state'].value_counts()
    for i in range(len(d.index)):
        states_count[ d.index[i] ] += d[i]

In [ ]:
################################################################
# Bar plot of number of states

fig, ax = plt.subplots(figsize=(15,5))

labels = [str((i, j)) for i in range(4) for j in range(4)]

ax.bar(x = labels, height = states_count.values())
ax.set_title('Number of states across all agents')

fig.tight_layout()
fig.savefig(root + folder + '/' +folder+ '_states_count.jpg', bbox_inches='tight', dpi=250)

In [ ]:
################################################################
# Correlations between actions and trips

corr = simulation_data['pcc']
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

labels = [action_legends[i] for i in range(5)]
labels.append('trips')

with sns.axes_style("dark"):
    f, ax = plt.subplots(figsize=(7, 5))
    ax = sns.heatmap(corr, cmap="YlGnBu", mask=mask, vmax=.3, square=True, annot=True, yticklabels=labels, xticklabels=labels)

plt.title('Correlation Coeffecients across all agents')
plt.savefig(root + folder + '/' +folder+ '_actions_trips_correlation.jpg', bbox_inches='tight', dpi=250)

In [ ]:
################################################################
# Transition Matrix Actions and Trips

plt.figure(figsize=(10,5))

temp = simulation_data['action_TM']['total']
sns_plot = sns.heatmap(temp, cmap="YlGnBu", annot=True, fmt="d", xticklabels=action_legends, yticklabels=action_legends)
plt.title('Transition Matrix for Actions across all agents')

plt.savefig(root + folder + '/' +folder+ '_actions_TM.jpg', bbox_inches='tight', dpi=250)

In [ ]:
################################################################
# State Action count

temp = simulation_data['state_action_TM']['total']

labels = []
for i in range(4):
    for j in range(4):
        labels.append((i,j))

plt.figure(figsize=(10,15))
sns_plot = sns.heatmap(temp, cmap="YlGnBu", annot=True, fmt="d", yticklabels=labels, xticklabels=action_legends)

plt.title('State Action Count Across all agents')
plt.savefig(root + folder + '/' +folder+ '_state_actions_TM.jpg', bbox_inches='tight', dpi=250)

In [ ]:
################################################################
# Transition Matrix States

temp = simulation_data['state_TM']['total']

labels = []
for i in range(4):
    for j in range(4):
        labels.append((i,j))

plt.figure(figsize=(20,15))
sns_plot = sns.heatmap(temp, cmap="YlGnBu", annot=True, fmt="d", yticklabels=labels, xticklabels=labels)

plt.title('Transition Matrix States across all agents')
plt.savefig(root + folder + '/' +folder+ '_state_TM.jpg', bbox_inches='tight', dpi=250)

In [ ]:
################################################################
# Actions and Trips Pair Plot

df = pd.DataFrame({
    'trips': simulation_data['total_binned_trips'],
    'Drop Home Pheromone': simulation_data['total_binned_actions'][0],
    'Drop Food Pheromone': simulation_data['total_binned_actions'][1],
    'Follow Home Pheromone': simulation_data['total_binned_actions'][2],
    'Follow Food Pheromone': simulation_data['total_binned_actions'][3],
    'Move Randomly': simulation_data['total_binned_actions'][4],
})

sns_plot = sns.pairplot(df, corner=False, kind='kde')
plt.title('pair Plot actions and trips across all agents')
plt.savefig(root + folder + '/' +folder+ '_pairplot.jpg', bbox_inches='tight', dpi=250)